<a href="https://colab.research.google.com/github/MNoichl/speech_to_text_with_whisper_to_GPT/blob/main/style_instruction_generator_MN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Style-instruction-generator 📚 🦜

*by [Max Noichl](https://homepage.univie.ac.at/maximilian.noichl/)*

This notebook uses gpt-4 to generate prompts/instructions that can be used to mimic a writing style. Just give it a writing sample, and it will spit out style instructions in response. I got the idea from this reddit [post](https://www.reddit.com/r/ChatGPT/comments/10rkqm0/ive_read_comments_asking_if_there_is_a_way_for/). It can make for entertaining literary experiments, but also quite useful. I use GPT to clean up dictations I make, and using the style prompts I can get the texts much closer to my own natural voice.

Importantly, for this to work, you will need an OpenAi-API-key, and pay for your usage. I feel this is quite reasonably priced, but until you get a feeliong for it, I suggest you monitor your spending! 

If you create a style you like, please add it to the repository:
[https://github.com/MNoichl/speech_to_text_with_whisper_to_GPT](https://github.com/MNoichl/speech_to_text_with_whisper_to_GPT)!

In [10]:
#@title Load dependencies
%%capture
!pip install openai
import time
import shutil
import os
from google.colab import drive
import openai
import pprint
import tqdm

In [11]:
#@markdown # Set up your API-key
#@markdown Here you have to provide your api-key to the OpenAI-API. 
#@markdown **WARNING: Do not share the notebook with your key in it (or in it's edit-history),
#@markdown because this will allow people to spend your money! 
#@markdown Also make sure to set reasonable spending limits at OpenAIs website,
#@markdown and monitor your spending regularly.** Alternatively you can
#@markdown  put a path to a text file in your drive, where you keep your key.

api_key_or_path = "" #@param {type:"string"}

# Check if api_key is a file
if os.path.isfile(api_key_or_path):
    # If it is a file, open it and read the content into a string
    with open(api_key_or_path, 'r') as file:
        api_key = file.read()
    openai.api_key = api_key # this tells the OpenAI library about your key, so it can be used below.
else:
    openai.api_key = api_key_or_path # this tells the OpenAI library about your key, so it can be used below.


In [12]:
#@title generate style instructions

text_sample =   "I have said that the visible product of Menard's pen is easily enumerated. Having examined his personal files with the greatest care, I have established that his body of work consists of the following pieces: a) a symbolist sonnet that appeared twice (with variants) in the review La Conque (in the numbers for March and October, 1899); b) a monograph on the possibility of constructing a poetic vocabulary from concepts that are neither synonyms nor periphrastic locutions for the concepts that inform common speech, \"but are, rather, ideal objects created by convention essentially for the needs of poetry\"(N\xEEmes,1901); c) a monograph on \"certain connections or affinities\" between the philosophies of Descartes, Leibniz, and John Wilkins(N\xEEmes,1903); d) a monograph on Leibniz' Characteristica universalis (N\xEEmes,1904); e) a technical article on the possibility of enriching the game of chess by eliminating one of the rook's pawns (Menard proposes, recommends, debates, and finally rejects this innovation); f) a monograph on Ramon Lull's Ars magna generalis (N\xEEmes,1906); g) a translation, with introduction and notes, of Ruy L\xF3pez de Segura's Libro de la invenci\xF3n liberal y arte del juego de la xedrez (Paris, 1907); h) drafts of a monograph on George Boole's symbolic logic; i) a study of the essential metrical rules of French prose, illustrated with examples taken from Saint-Simon (Revuedes langues romanes, Montpellier, October 1909); j) a reply to Luc Durtain (who had countered that no such rules existed), illustrated with examples taken from Luc Durtain (Revuedes langues romanes, Montpellier, December 1909); k) a manuscript translation of Quevedo's Aguja de navegar cultos, titled La boussole des precious; l) a foreword to the catalog of an exhibit of lithographs by Carolus Hourcade(N\xEEmes,1914); m) a work entitled Les probl\xE8mes d'un probl\xE8me (Paris,1917), which discusses in chronological order the solutions to the famous problem of Achilles and the tortoise (two editions of this work have so far appeared; the second bears an epigraph consisting of Leibniz' advice \"Ne craignez point, monsieur, la tortue,\" and brings up to date the chapters devoted to Russell and Descartes); n) a dogged analysis of the \"syntactical habits\" of Toulet (N.R.F., March 1921) (Menard, I recall, affirmed that censure and praise were sentimental operations that bore not the slightest resemblance to criticism); o) a transposition into alexandrines of Paul Val\xE9ry's Cimeti\xE8re marin (N.R.F., January 1928); p) a diatribe against Paul Val\xE9ry, in Jacques Reboul's Feuilles pour la suppression de la r\xE9alit\xE9 (which diatribe, I might add parenthetically, states the exact reverse of Menard's true opinion of Val\xE9ry; Val\xE9ry understood this, and the two men's friendship was never imperiled); q) a \"definition\" of the countess de Bagnoregio, in the \"triumphant volume\" (the phrase is that of another contributor, Gabriele d'Annunzio) published each year by that lady to rectify the inevitable biases of the popular press and to present \"to the world and all of Italy\" a true picture of her person, which was so exposed (by reason of her beauty and her bearing) to erroneous and/or hasty interpretations; r) a cycle of admirable sonnets dedicated to the baroness de Bacourt(1934); s) a handwritten list of lines of poetry that owe their excellence to punctuation.1 [ 1 Mme. Henri Bachelier also lists a literal translation of Quevedo's literal translation of St. Francis de Sales's Introduction \xE0 la vie d\xE9vote. In Pierre Menard's library there is no trace of such a work. This must be an instance of one of our friend's droll jokes, misheard or misunderstood.] This is the full ext" #@param {type:"string"}

language = 'en' #@param ["en", "de"] {allow-input: true}

OpenAi_model = "gpt-4" #@param ["gpt-4", "gpt-3.5-turbo"] {allow-input: true}




def send_texts_to_gpt_api(texts,model='gpt-4'):
    messages = []
    last_reply = None
    messages.append({"role": "system", "content": system_prompt})
    
    for text in tqdm.tqdm_notebook(texts):

        messages.append({"role": "user", "content": text})
        # print(messages)
        response = openai.ChatCompletion.create(
            model=model,
            messages=messages,
        )
        
        reply = response['choices'][0]['message']['content']
        messages.append({"role": "system", "content": reply})


        last_reply = reply
        
    return last_reply




if language == 'en':


  system_prompt = """You are a linguistics expert who is an expert in HSC English Common Module: Texts and Human Experiences and an expert in the study of modern language."""
  create_analysis_tool = """Create a list of 25 techniques that you would use to analyze a sample of writing for the purpose of creating a comprehensive, systematic, objective, precise, writing-style profile that is unique to that sample. In addition, your list will also include your techniques in using human linguistic systems, such as syntax, semantics, morphology, phonology and pragmatics to analyze the sample of writing for the purpose of creating a comprehensive, systematic, objective, precise, writing-style profile that is unique to that sample. The name of the list is “List A”."""
  apply_analysis_tool = """Use all 25 analysis techniques in List A to analyze the sample text in parentheses to create an analysis of the technical aspect of the sample text. Call this analysis Profile A ("""
  rephrase_to_prompt = """Rephrase each of the 25 analysis, in Profile A, into writing-style-instructions, without referencing the text, that I can use in a prompt that will make you use the writing-style to generate text. Name the writing-style-instructions Writing style A"""
  rephrase_to_text = """Nice! Format it now into a block of text!"""

  texts = [create_analysis_tool,
          apply_analysis_tool + text_sample + ')',
          rephrase_to_prompt,
          rephrase_to_text]
  result = send_texts_to_gpt_api(texts, model=OpenAi_model)

elif  language == 'de':
  system_prompt = """Du bist ein Sprachwissenschaftler der sich mit dem Studium der modernen Sprache auskennt."""
  create_analysis_tool = """Erstelle eine Liste von 25 Techniken, die du verwenden würdest, um eine Schreibprobe zu analysieren, um ein umfassendes, systematisches, objektives, präzises, auf diese Probe einzigartiges Schreibstilprofil zu erstellen. Darüber hinaus enthält Ihre Liste auch Ihre Techniken zur Verwendung menschlicher Sprachsysteme, wie Syntax, Semantik, Morphologie, Phonologie und Pragmatik, um die Schreibprobe zu analysieren, um ein umfassendes, systematisches, objektives, präzises Schreibstilprofil zu erstellen, das auf diese Probe einzigartig ist. Der Name der Liste ist "Liste A"."""
  apply_analysis_tool = """Verwende alle 25 Analysetechniken in Liste A, um den Text in Klammern zu analysieren und eine Analyse des technischen Aspekts des Textes zu erstellen. Nennen Sie diese Analyse Profil A ("""
  rephrase_to_prompt = """Formuliere jede der 25 Analysen in Profil A um in Schreibstil-Anweisungen, ohne den Text zu nennen, damit ich sie in einer Aufforderung verwenden kann, die dich dazu bringt, den Schreibstil zum Generieren von Text zu verwenden. Nenne die Schreibstil-Anweisungen Schreibstil A"""
  rephrase_to_text = """Super! Formatiere es jetzt zu einem Textblock!"""

  texts = [create_analysis_tool,
          apply_analysis_tool + text_sample + ')',
          rephrase_to_prompt,
          rephrase_to_text]
  result = send_texts_to_gpt_api(texts)



pp = pprint.PrettyPrinter(indent=4)
pp.pprint(result)


<ipython-input-12-edd9cde12fd8>:17: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for text in tqdm.tqdm_notebook(texts):


  0%|          | 0/4 [00:00<?, ?it/s]

('To emulate Writing Style A, incorporate a range of specialized terms and '
 'proper nouns relevant to the subject matter. Construct sentences primarily '
 'with a subject-verb-object structure but include multiple parenthetical '
 'phrases for complexity. Focus on literal meanings of technical terms and '
 'phrases, limiting the use of figurative language. Employ compound words and '
 "various inflectional forms for diversity, while assuming the reader's "
 'familiarity with the subject matter and referenced figures. Use limited '
 'transitional devices and logical organization, and rely predominantly on '
 'past tense, with occasional present tense verbs. Maintain an authoritative '
 'and scholarly tone throughout, refraining from incorporating dialogue. \n'
 '\n'
 'Employ a narratorial voice with intimate knowledge of the subject matter, '
 'incorporating themes such as academic production, authorship, and '
 'intellectual debate. Utilize frequent parentheses to establish patterns 